In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import ipywidgets as widgets
%matplotlib widget


def adjust_spines(ax, spines):
    for loc, spine in ax.spines.items():
        if loc in spines:
            spine.set_position(('outward', 10))  # outward by 10 points
        else:
            spine.set_color('none')  # don't draw spine

    # turn off ticks where there is no spine
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        # no yaxis ticks
        ax.yaxis.set_ticks([])

    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
        # no xaxis ticks
        ax.xaxis.set_ticks([])

def nearest(arr, val):
    idx = np.argmin(np.abs(arr - val))
    return idx, arr[idx]

def unispace(start, stop, step):
    a, b = (np.ceil(start/step), np.floor(stop/step))
    return np.arange(a, b+1) * step

In [2]:
_a, _n, _x0, _y0 = 1, 1, 0, 0
_x1, _x2, _dx = -5, 5, 0.1
_y1, _y2 = -5, 5

def f(x, a=_a, n=_n, x0=_x0, y0=_y0):
    if n > 0:
        return y0 + a * (x - x0)**n
    elif n == 0:
        return y0 + a * np.ones(x.size)
    else:
        y = np.full(x.size, np.nan)
        i, _ = nearest(x, x0)
        mask = np.arange(len(y)) != i
        y[mask] = y0 + a * (x[mask] - x0)**n
        y[i] = np.nan
        return y  

x = unispace(_x1, _x2, _dx)

In [3]:
out = widgets.Output()
with out:
    fig = plt.figure(figsize=(5, 5), constrained_layout=True)
    axs = fig.add_subplot()
    plt.show(fig)
    
fig.canvas.toolbar_position = 'right'
fig.canvas.header_visible = False

axs.set_xlim(_x1, _x2)
axs.set_ylim(_y1, _y2)
axs.set_aspect('equal')

adjust_spines(axs, ['left', 'bottom'])
axs.set_xlabel(r'$x$')
axs.set_ylabel(r'$y$')

axs.grid(True)
axs.axhline(0, color='grey', ls='-', lw=0.8)
axs.axvline(0, color='grey', ls='-', lw=0.8)

line, = axs.plot(x, f(x))

In [5]:
slider_a = widgets.FloatSlider(value=_a, min=-4, max=4, step=0.2, continuous_update=False, description=r'\(a\)', layout=widgets.Layout(width='400px'))
slider_n = widgets.IntSlider(value=_n, min=-8, max=8, step=1, continuous_update=False, description=r'\(n\)', layout=widgets.Layout(width='400px'))
slider_x0 = widgets.FloatSlider(value=_x0, min=-4, max=4, step=0.2, continuous_update=False, description=r'\(x_0\)', layout=widgets.Layout(width='400px'))
slider_y0 = widgets.FloatSlider(value=_y0, min=-4, max=4, step=0.2, continuous_update=False, description=r'\(y_0\)', layout=widgets.Layout(width='400px'))

slider_x12 = widgets.FloatRangeSlider(value=[_x1, _x2], min=-10, max=10, step=0.5, continuous_update=False, description=r'\(x\)', layout=widgets.Layout(width='400px'))
slider_y12 = widgets.FloatRangeSlider(value=[_y1, _y2], min=-10, max=10, step=0.5, continuous_update=False, description=r'\(y\)', layout=widgets.Layout(width='400px'))
slider_dx = widgets.FloatLogSlider(value=_dx, base=10, min=-3, max=0, step=1/4, continuous_update=False, description=r'\(\mathrm{d}x\)', layout=widgets.Layout(width='400px'))

def up_data(_):
    x = unispace(slider_x12.value[0], slider_x12.value[1], slider_dx.value)
    line.set_data(x, f(x, a=slider_a.value, n=slider_n.value, x0=slider_x0.value, y0=slider_y0.value))
    # fig.canvas.draw()
    
def up_xlim(_):
    axs.set_xlim(slider_x12.value[0], slider_x12.value[1])
    up_data(_)
    
def up_ylim(_):
    axs.set_ylim(slider_y12.value[0], slider_y12.value[1])
    # fig.canvas.draw()

slider_a.observe(up_data)
slider_n.observe(up_data)
slider_x0.observe(up_data)
slider_y0.observe(up_data)
slider_x12.observe(up_xlim)
slider_dx.observe(up_xlim)
slider_y12.observe(up_ylim)

label = widgets.Label(r'\(y = f(x) = a \cdot (x - x_0)^n + y_0\)')
contr_params = widgets.VBox([slider_a, slider_n, slider_x0, slider_y0])
contr_lims = widgets.VBox([slider_x12, slider_dx, slider_y12])
controls = widgets.Accordion(children=[contr_params, contr_lims])

widgets.HBox([widgets.VBox([label, controls]), out])